<h1><center>Fanlore.org scraping project<center>
<center>DSBA course<center>
<center>Organized by Karol Rochalski<center><h1>

In [171]:
from bs4 import BeautifulSoup
import pathlib
import os
import pandas as pd
import numpy as np
import regex as re
import requests
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import shutil

Url of website we are going to scrape is stored in external .txt file. 

This is conducted for safety reasons.

We access the file and read the first line, then assign it to the variable.

In [3]:

file = open('./urls.txt', 'r')
url = file.readline()
url

'https://fanlore.org/wiki/Category:Fan_Communities'

In [4]:
driver = webdriver.Chrome()
driver.quit()

Default request to crawl the website. 

It takes url from .txt file and operates on them in order to find certain data.

Below that new variable 'soup' has standard BeautifulSoup parser.

In [5]:

page = requests.get(url)

soup = BeautifulSoup(page.text, 'html.parser')


Outputs content of the website. This is not desired format. In fact it's barely readable.

The big advantage of it is that we get a good grasp of what we are going to work with and 

confirm the website is responsive. It selves as a good starting point.

In [6]:
print(soup.title)
print(soup.prettify())

<title>Category:Fan Communities - Fanlore</title>
<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Category:Fan Communities - Fanlore
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"9e9a34cc381184c96c965433","wgCSPNonce":!1,"wgCanonicalNamespace":"Category","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":14,"wgPageName":"Category:Fan_Communities","wgTitle":"Fan Communities","wgCurRevisionId":2508947,"wgRevisionId":2508947,"wgArticleId":382,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Fanlore Categories"],"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgRelev

<h2>Testing basics</h2>

The main format of the project consists of groups of scraped values put into numpy arrays. 

The project's final output will summarize dataframes utilizing main sections of the website: Subcategories and Fan 

Communities with regard to their information (names, links, sections). The other data will provide scraped images and 

other minor data.

In [7]:
div = soup.find_all('div', class_='')
divs = []
for x in div:
    divs.append(str(x))
#divs.append(str(div))
divsNP = np.array(divs)

<h2>Captcha Resolver</h2>

Dismissed in process of work.

In [173]:
driver = webdriver.Chrome()

driver.get(url)

driver.quit()

<h2>Subcategories</h2>

In this part we are extracting Subcategory information. The main goal is to get titles and external links.

We extract the formula from the div, proceed to the span tag and then extract titles from data-ct-title attribute.

In [9]:

#Subcategories
Subcategory =  soup.find('div', id ='mw-subcategories').findAll('a')

Subcategories = []
for x in Subcategory:
    attr = x.attrs

    n = attr.get('title')

    #slicing beginning of the string to avoid mess
    n = n[9:]

    Subcategories.append(n)

SubcategoriesNPTitles = np.array(Subcategories)
print(SubcategoriesNPTitles[0])
print(type(SubcategoriesNPTitles[0]))


Communities by Fandom
<class 'numpy.str_'>


For links we are gonna copy-paste same code and modify it to extract the links.

In [10]:
Subcategory =  soup.find('div', id ='mw-subcategories').findAll('a')

Subcategories = []
for x in Subcategory:
    attr = x.attrs
    #print(attr)
    Subcategories.append(attr.get('href'))

SubcategoriesNPLinks = np.array(Subcategories)
print(SubcategoriesNPLinks.size)
for x in SubcategoriesNPLinks:
    print(x)
print(type(SubcategoriesNPLinks[1]))

17
/wiki/Category:Communities_by_Fandom
/wiki/Category:Communities_by_Year
/wiki/Category:Discord_Servers
/wiki/Category:Fandoms_by_Source_Text
/wiki/Category:Fandoms_by_Canon_Type
/wiki/Category:Fandoms_by_Community
/wiki/Category:Forums
/wiki/Category:Imageboards
/wiki/Category:Journaling_Communities
/wiki/Category:Mailing_Lists
/wiki/Category:Microblogs
/wiki/Category:Multimedia_Communities
/wiki/Category:Newsgroups
/wiki/Category:Offline_Spaces
/wiki/Category:Social_Networking
/wiki/Category:Subreddits
/wiki/Category:Websites
<class 'numpy.str_'>


And extract the corresponding number of child categories, pages and files.

In [58]:
childObject = soup.find('div', id ='mw-subcategories').findAll('span', dir='ltr')

SubcategoriesChildren = []

for x in childObject:
    attr = x.attrs
    seq = attr.get('title')
    seq = re.sub(r',', '', seq)
    sub = re.findall(r'[0-9]+', seq)
    SubcategoriesChildren.append(sub)


SubcategoriesNPChildren = np.array(SubcategoriesChildren)
print(SubcategoriesNPChildren[:, 0])

['28' '39' '2' '1484' '21' '75' '5' '2' '18' '27' '2' '0' '4' '6' '5' '0'
 '85']


We can also check if the element is expandable.

In this particular example we do not loop through attributes of the element. 

Instead we check if there's class called 'CategoryTreeEmptyBullet'. If so, we append

False to the list, else True.

In [79]:
Expandable = soup.find('div', id ='mw-subcategories').findAll('div', class_='CategoryTreeItem')

Expandables = []
#.find('span', class_='CategoryTreeEmptyBullet')
for x in Expandable:
    if x.find('span', class_='CategoryTreeEmptyBullet'):
        Expandables.append(False)
    else:
        Expandables.append(True)

ExpandablesNP = np.array(Expandables)
print(ExpandablesNP)
print(len(ExpandablesNP))


[ True  True  True  True  True  True  True  True  True  True  True False
  True  True  True False  True]
17


<h2>First Dataframe</h2>

Now we can combine columns into single dataframe.

In [161]:
column_name = ['Titles', 'Links', 'Nr. of child categories', 'Nr. of child Pages', 'Nr. of child files', 'Expandable']
df = pd.DataFrame({column_name[0]: SubcategoriesNPTitles, column_name[1]: SubcategoriesNPLinks,
                  column_name[2]: SubcategoriesNPChildren[:, 0], column_name[3]: SubcategoriesNPChildren[:, 1], column_name[4]: SubcategoriesNPChildren[:, 2], column_name[5]: ExpandablesNP})
df

,Titles,Links,Nr. of child categories,Nr. of child Pages,Nr. of child files,Expandable
0,Communities by Fandom,/wiki/Category:Communities_by_Fandom,28,0,0,True
1,Communities by Year,/wiki/Category:Communities_by_Year,39,0,0,True
2,Discord Servers,/wiki/Category:Discord_Servers,2,30,0,True
3,Fandoms by Source Text,/wiki/Category:Fandoms_by_Source_Text,1484,1074,0,True
4,Fandoms by Canon Type,/wiki/Category:Fandoms_by_Canon_Type,21,53,0,True
5,Fandoms by Community,/wiki/Category:Fandoms_by_Community,75,23,1,True
6,Forums,/wiki/Category:Forums,5,146,29,True
7,Imageboards,/wiki/Category:Imageboards,2,2,0,True
8,Journaling Communities,/wiki/Category:Journaling_Communities,18,1157,21,True
9,Mailing Lists,/wiki/Category:Mailing_Lists,27,813,0,True


FInally we can save it to the csv file in separate folder.

In [162]:
df.to_csv('./dataframes/Sub_Communities', encoding='utf-8')

<h2> Fan Communities data </h2>

In [101]:
#Fan Communities listed
comm = soup.find('div', id="mw-pages")
for x in comm:
    print(x)



<h2>Pages in category "Fan Communities"</h2>


<p>The following 180 pages are in this category, out of 180 total.
</p>
<div class="mw-content-ltr" dir="ltr" lang="en"><div class="mw-category"><div class="mw-category-group"><h3>2</h3>
<ul><li><a href="/wiki/2_Builders_2_Tools" title="2 Builders 2 Tools">2 Builders 2 Tools</a></li></ul></div><div class="mw-category-group"><h3>A</h3>
<ul><li><a href="/wiki/A_A_Sonic_Shipping" title="A A Sonic Shipping">A A Sonic Shipping</a></li>
<li><a href="/wiki/Aarinfantasy" title="Aarinfantasy">Aarinfantasy</a></li>
<li><a href="/wiki/TheAD%27sOffice" title="TheAD'sOffice">TheAD'sOffice</a></li>
<li><a href="/wiki/Akte_X_Fans" title="Akte X Fans">Akte X Fans</a></li>
<li><a href="/wiki/Anti-ZADR-Club" title="Anti-ZADR-Club">Anti-ZADR-Club</a></li>
<li><a href="/wiki/Arrows_Online" title="Arrows Online">Arrows Online</a></li>
<li><a href="/wiki/As_Others_See_Us:_Fanfic_in_the_Media" title="As Others See Us: Fanfic in the Media">As Others See Us: Fan

The 'div' element is responding successfully,

We can start constructing this model by using scheme created in first part of the previous example. 

TL;DR We will get titles, their corresponding links and literary category they belong to.

In [163]:
titleC = soup.find('div', id="mw-pages").find_all('a')

titlesC = []

for x in titleC:
    attr = x.attrs
    n = attr.get('title')
    titlesC.append(n)

TitlesCNP = np.array(titlesC)
for x in TitlesCNP:
    print(x)

2 Builders 2 Tools
A A Sonic Shipping
Aarinfantasy
TheAD'sOffice
Akte X Fans
Anti-ZADR-Club
Arrows Online
As Others See Us: Fanfic in the Media
Assassin's Creed Sisterhood
The Balance
Baraag
Bay City Library
Berries.space
Beyond Atlantis Virtual Season Six
Big Light Network
Bit of Earth
Blame Taylor (Facebook group)
Bloody Awful Poet Society
Brickfilming
Bring It Fanfics
Buffistas.org
Buffyverse Music Video Database
C-Leapers
Camelot Music Video Awards
Casa de Ideas
Celebrity Marriage Clique
A Change of Hobbit
Chaos.x-philes.com
Christian Angels
Christian Fan Fellowship
Comic Fan-Fiction Authors' Network
Corrie's Unhappy Father Todd To Be Snared
Cotton Candy (Hanson fan club)
DailyVelCinta
Dawnsisters Weyr
DC-Slash
Doctor Who Fan Orchestra
Donaldism
Doppelleben
DSE
Duck Prints Press
Due SOUTH Slash Virtual Season
Es de Fanfics
Fake F1
Fallen Grace's Tavern
Fan 911 Auction
Fan Fiction University
Fan Soundtrack Project
Fan Studies Network
Mary Sue
Fandom Content on Mainstream Spaces
Fand

In [164]:
linkC = soup.find('div', id="mw-pages").find_all('a')
print(linkC.__len__())
linksC = []

for x in linkC:
    attr = x.attrs
    print(x.parent)
    linksC.append(attr.get('href'))

linksCNP = np.array(linksC)
for x in linksCNP:
    print(x)

180
<li><a href="/wiki/2_Builders_2_Tools" title="2 Builders 2 Tools">2 Builders 2 Tools</a></li>
<li><a href="/wiki/A_A_Sonic_Shipping" title="A A Sonic Shipping">A A Sonic Shipping</a></li>
<li><a href="/wiki/Aarinfantasy" title="Aarinfantasy">Aarinfantasy</a></li>
<li><a href="/wiki/TheAD%27sOffice" title="TheAD'sOffice">TheAD'sOffice</a></li>
<li><a href="/wiki/Akte_X_Fans" title="Akte X Fans">Akte X Fans</a></li>
<li><a href="/wiki/Anti-ZADR-Club" title="Anti-ZADR-Club">Anti-ZADR-Club</a></li>
<li><a href="/wiki/Arrows_Online" title="Arrows Online">Arrows Online</a></li>
<li><a href="/wiki/As_Others_See_Us:_Fanfic_in_the_Media" title="As Others See Us: Fanfic in the Media">As Others See Us: Fanfic in the Media</a></li>
<li><a href="/wiki/Assassin%27s_Creed_Sisterhood" title="Assassin's Creed Sisterhood">Assassin's Creed Sisterhood</a></li>
<li><a href="/wiki/The_Balance" title="The Balance">The Balance</a></li>
<li><a href="/wiki/Baraag" title="Baraag">Baraag</a></li>
<li><a href=

<h2>Corresponding category</h2>

Now we can add categories to the project to better construct the dataframe.

In [165]:
CategoryC = soup.find('div', id="mw-pages").find_all('a')

CategoriesC = []
for x in linkC:
    p = x.parent.parent.parent.text

    CategoriesC.append(p[0])

CategoriesCNP = np.array(CategoriesC)
for x in CategoriesCNP:
    print(x)

2
A
A
A
A
A
A
A
A
B
B
B
B
B
B
B
B
B
B
B
B
B
C
C
C
C
C
C
C
C
C
C
C
D
D
D
D
D
D
D
D
D
E
F
F
F
F
F
F
F
F
F
F
F
F
F
F
F
F
F
F
F
F
F
F
F
G
G
G
G
G
G
G
H
H
H
H
H
H
H
H
H
I
I
I
I
I
I
J
K
K
K
L
L
L
L
M
M
M
M
M
M
M
M
N
N
O
O
P
P
P
P
P
P
P
R
R
R
R
R
R
R
R
S
S
S
S
S
S
S
S
S
S
S
S
S
S
S
S
S
S
S
S
S
S
S
T
T
T
T
T
T
T
T
T
T
T
T
T
T
U
V
V
V
V
W
W
W
W
W
W
W
X
X
X
X
X
X
X
Z


<h2>Communities Dataset</h2>

In [166]:
column_name = ['Titles', 'Links', 'Parent Category']
df2 = pd.DataFrame({column_name[0]: TitlesCNP, column_name[1]: linksCNP, column_name[2]: CategoriesCNP})

#x = map(lambda x: int(x) + int(x), stre.strip().split(' '))

df2['Sum of Characters'] = df.apply(lambda row: int(len(row['Titles'])), axis=1)
df2

,Titles,Links,Parent Category,Sum of Characters
0,2 Builders 2 Tools,/wiki/2_Builders_2_Tools,2,21.0
1,A A Sonic Shipping,/wiki/A_A_Sonic_Shipping,A,19.0
2,Aarinfantasy,/wiki/Aarinfantasy,A,15.0
3,TheAD'sOffice,/wiki/TheAD%27sOffice,A,22.0
4,Akte X Fans,/wiki/Akte_X_Fans,A,21.0
...,...,...,...,...
175,The X-Files University,/wiki/The_X-Files_University,X,NaN
176,X-Ville,/wiki/X-Ville,X,NaN
177,XFiles-Mania,/wiki/XFiles-Mania,X,NaN
178,XFilesNews,/wiki/XFilesNews,X,NaN


Save dataframe

In [167]:
df2.to_csv('./dataframes/Fan_Communities', encoding='utf-8')

<h2> Image extraction </h2>

Now it's time for the funniest part - image extraction. Even though we don't work with fan subscription data, there's still a lot to take in consideration.

In [98]:
folder_name = 'images'
dir = os.listdir(folder_name)

images = soup.find_all('img')

if len(dir) == 0:
    for i, image in enumerate(images):
        image_link = image['src']
        print(image_link)
        full_url = url + image_link
        r = requests.get(full_url).content
        #r = str(r, 'utf-8')
        with open(f"{folder_name}/image{i + 1}.png", 'wb+') as f:
            f.write(r)
else:
    print('Folder has already been populated')


/w/images/thumb/2/20/Christianfanfellowship.jpg/92px-Christianfanfellowship.jpg
/w/images/thumb/d/d3/Ficwip_logo.png/120px-Ficwip_logo.png
/w/images/thumb/0/08/Genieflyer.jpg/92px-Genieflyer.jpg
/w/images/thumb/e/e0/Idealhaven.jpg/120px-Idealhaven.jpg
/w/images/thumb/7/70/Ladies_Sewing_Circle.png/120px-Ladies_Sewing_Circle.png
/w/images/thumb/8/8c/Sith_Academy-the_Original_Illustration_%281999%29.jpg/120px-Sith_Academy-the_Original_Illustration_%281999%29.jpg
/w/images/thumb/b/bb/Welcome_to_the_Sith_Academy2.png/120px-Welcome_to_the_Sith_Academy2.png
/w/skins/by-nc.png
/w/resources/assets/poweredby_mediawiki_88x31.png
